In [3]:
import nasdaqdatalink
from fredapi import Fred
from statsmodels.tsa.stattools import adfuller,grangercausalitytests
from pandas_datareader.data import DataReader
from datetime import date, datetime
import pandas as pd
import numpy as np

nasdaqdatalink.ApiConfig.api_key = 'rS47bzkfYmSAV-hk-4-w'
fred = Fred(api_key='7e70df3f020852ea824275ce0fea70a0')

## Multivariate Data

In [2]:
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')

# monthly CPI


macro = [
         'INDPRO', # industrial production, Jan 1919 to Aug 2022
         'CPIAUCSL', # CPI, 01/57 to 08/2022
          'DFF' # federal funds rate, 1954-07-01 to 2022-10-13
         ] 
data = DataReader(macro,"fred" ,start=datetime(1957, 1, 1), end=datetime(2022, 8, 1))
         
data.columns = ['indpro', 'cpi', 'ffr']  

In [12]:
# gather the data for the first day of each month to store the monthly data from daily data
# groupby the month and then get the zeroth (nth) row of each group.
data.sort_index(inplace=True)
data_grouped = data.groupby(pd.Grouper(freq ='M')).nth(0)

In [13]:
# calculate the percent change from 1 year (or 12 months ago) ago
rate = data_grouped[['indpro', 'cpi']].pct_change(12).add_suffix("_rate").dropna()
data_change = rate*100
short = data_grouped[['ffr']]
data_change = pd.concat([data_change, short], axis=1).dropna()

In [14]:
data_change.tail()

,indpro_rate,cpi_rate,ffr
DATE,,,
2022-04-30,5.297976,8.224139,0.33
2022-05-31,4.416082,8.516413,0.33
2022-06-30,3.898403,8.995221,0.83
2022-07-31,3.945777,8.482130,1.58
2022-08-31,3.909974,8.249235,2.33


In [15]:
data_change.to_excel("data_change.xlsx",index= True, header= True)

## Data for Example 2 

In [16]:
# daily values

macro2 = [
         'DFF', # Federal funds rate, 1954-07-01 to 2022-10-13
         'DGS10', # 10 year Treasury yield, 1962-01-02 to 2022-10-13 
         'UNRATE', # Unemployment rate, Jan 1948 to Sep 2022
         'CPIAUCSL', # CPI, Jan 1947 to Sep 2022
         ] 

# as some variables have daily data, such as ffr, and 10 year Treasury yield, 
# data2 will have daily observations for these two, with NaN for the monthly variables
data2 = DataReader(macro2,"fred" ,start=datetime(1962, 1, 1), end=datetime(2022, 9, 1))
         
data2.columns = ['ffr', 'tres10y', 'unrate', 'cpi']  

In [17]:
# convert daily to monthly values by taking the mean of the values in the month (for daily data)
data2.index = pd.to_datetime(data2.index)
data_grouped2 = data2.resample('1M').mean() # stores monthly values

# calculate the percent change from 1 year (or 12 months ago) ago, 
# and rename the variable to cpi_rate to reflect that the variable is percent change and not level
rate2 = data_grouped2[['cpi']].add_suffix('_rate').pct_change(12).dropna()
data_change2 = rate2*100

short = data_grouped2[['ffr','tres10y', 'unrate']] # gather the remaining data 

# merge or concatenate the remaining (raw) data with the percent change (or rates) variables
data_change2 = pd.concat([short, data_change2], axis=1).dropna()
data_change2.to_excel("data_four.xlsx",index= True, header= True)

## Data for Example 4

In [18]:
# daily values

macro4 = [
         'DFF', # Federal funds rate, 1954-07-01 to 2022-10-13
         'INDPRO', # industrial production, Jan 1919 to Aug 2022
         'CPIAUCSL', # CPI, Jan 1947 to Sep 2022
         'A794RX0Q048SBEA', #  Real personal consumption expenditures per capita ($), Q1 1947 to Q2 2022
         'UNRATE', # Unemployment rate, Jan 1948 to Sep 2022
          'GPDIC1',# Real Gross Private Domestic Investment, Q1 1947 to Q2 2022
         'NETEXP', # Net Exports of Goods and Services, Q1 1947 to Q2 2022
         'M2REAL', # Real M2 Money Stock (billions of $), Jan 1959 to Aug 2022
         'DGS10', # 10 year Treasury yield, 1962-01-02 to 2022-10-13 
         'NASDAQCOM', # NASDAQ Composite Index, 1971-02-05 to 2022-10-14
         'DEXJPUS' # Japanese yen to USD Spot exchange rate, 1971-01-04 to 2022-10-07
   ] 

# as some variables have daily data, such as ffr, and 10 year Treasury yield, 
# data2 will have daily observations for these two, with NaN for the monthly variables
data4 = DataReader(macro4,"fred" ,start=datetime(1971, 1, 1), end=datetime(2022, 9, 1))
         
data4.columns = ['ffr','indpro','cpi','rpce', 'unrate','gpdi','netexp','m2real', 'tres10y', 'nasdaq', 'yen_usd'] 
data4.head()

,ffr,indpro,cpi,rpce,unrate,gpdi,netexp,m2real,tres10y,nasdaq,yen_usd
DATE,,,,,,,,,,,
1971-01-01,3.0,37.6015,39.9,15298.0,5.9,620.212,4.57,1586.2,NaN,NaN,NaN
1971-01-02,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-03,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-04,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.46,NaN,357.73
1971-01-05,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.47,NaN,357.81


In [19]:
# gather the data for the first day of each month to store the monthly data from daily data
# groupby the month and then get the zeroth (nth) row of each group.
data4.sort_index(inplace=True)
data4_grouped = data4.groupby(pd.Grouper(freq ='Q')).nth(0)

# spline interpolation method to fill the NaN
inter_data4 = data4_grouped.interpolate(method='spline', order =3)
inter_data4.head()

,ffr,indpro,cpi,rpce,unrate,gpdi,netexp,m2real,tres10y,nasdaq,yen_usd
DATE,,,,,,,,,,,
1971-03-31,3.00,37.6015,39.9,15298.0,5.9,620.212,4.570,1586.2,NaN,NaN,NaN
1971-06-30,4.00,37.6998,40.1,15394.0,5.9,637.811,-0.401,1641.9,5.58000,106.340000,357.490000
1971-09-30,5.25,37.9395,40.6,15470.0,6.0,645.549,0.236,1673.9,6.69000,108.470000,357.400000
1971-12-31,5.38,38.6193,40.9,15678.0,5.8,628.165,-1.920,1707.6,6.00000,109.580000,333.390000
1972-03-31,3.00,40.1688,41.2,15847.0,5.8,669.558,-3.534,1742.0,6.02325,112.612247,309.596983


In [20]:
# calculate the percent change from 1 year (or 12 months ago) ago, 
# and rename the variable to cpi_rate to reflect that the variable is percent change and not level
rate2 = inter_data4[['indpro','cpi', 'rpce', 'gpdi', 'm2real', 'nasdaq', 'yen_usd']].add_suffix('_rate').pct_change(12).dropna()
data_change4 = rate2*100
data_change4.head()

,indpro_rate,cpi_rate,rpce_rate,gpdi_rate,m2real_rate,nasdaq_rate,yen_usd_rate
DATE,,,,,,,
1974-06-30,20.325572,19.950125,8.113551,17.132191,10.530483,-13.579086,-21.645361
1974-09-30,20.505805,21.428571,7.769877,9.730168,6.804469,-30.496912,-20.170677
1974-12-31,16.783836,24.694377,4.515882,13.451243,2.576716,-49.370323,-10.570203
1975-03-31,3.251777,26.941748,4.057550,-10.707362,-0.522388,-46.927914,-4.505627
1975-06-30,-2.696919,27.710843,3.792675,-18.022044,-0.843028,-34.612087,-0.170873


In [21]:
short4 = inter_data4[['ffr', 'unrate','netexp','tres10y']] # gather the remaining data 

# merge or concatenate the remaining (raw) data with the percent change (or rates) variables
data_change4 = pd.concat([short4, data_change4], axis=1).dropna()
data_change4.head()

,ffr,unrate,netexp,tres10y,indpro_rate,cpi_rate,rpce_rate,gpdi_rate,m2real_rate,nasdaq_rate,yen_usd_rate
DATE,,,,,,,,,,,
1974-06-30,10.01,5.1,-2.714,7.420000,20.325572,19.950125,8.113551,17.132191,10.530483,-13.579086,-21.645361
1974-09-30,14.33,5.5,-6.972,7.650000,20.505805,21.428571,7.769877,9.730168,6.804469,-30.496912,-20.170677
1974-12-31,11.30,6.0,0.033,7.970000,16.783836,24.694377,4.515882,13.451243,2.576716,-49.370323,-10.570203
1975-03-31,3.87,8.1,16.481,7.364891,3.251777,26.941748,4.057550,-10.707362,-0.522388,-46.927914,-4.505627
1975-06-30,5.48,8.8,21.582,8.080000,-2.696919,27.710843,3.792675,-18.022044,-0.843028,-34.612087,-0.170873


In [22]:
rearranged = data_change4.loc[:,['ffr','indpro_rate','cpi_rate', 'rpce_rate', 'unrate', 'gpdi_rate', 'netexp', 
                   'm2real_rate', 'tres10y', 'nasdaq_rate', 'yen_usd_rate']]
rearranged.to_excel("data_eleven.xlsx",index= True, header= True)

## Data for Example 4: FAVAR

In [5]:
# daily values

macro4 = [
         'FGEXPND', # Federal Government: Current Expenditures, Q1 1947 to Q2 2022
         'HOUST', # New Privately-Owned Housing Units Started: Total Units, Jan 1959 to Aug 2022
         'DCOILBRENTEU', # Crude Oil Prices: Brent - Europe, 1987-05-20 to 2022-10-11
         'WTISPLC', # Spot Crude Oil Price: West Texas Intermediate (WTI), Jan 1946 to Sep 2022
         'CCSA', # Continued Claims (Insured Unemployment), 1967-01-07 to 2022-10-01
         'GDPC1', # Real gross domestic product, Q1 1947 to Q2 2022
         'MANEMP', # All Employees, Manufacturing, Jan 1939 to Sep 2022
         'CES4300000001', # All Employees, Transportation and Warehousing, Jan 1972 to Sep 2022
         'INDPRO', # Industrial Production Index, Jan 1919 to Sep 2022
         'MORTGAGE30US', # 30-year fixed mortgage average, 1971-04-02 to 2022-10-13
         'USGOOD', # All Employees, Goods-Producing, Jan 1939 to Sep 2022
    
    # Industrial Production
    
         'IPUTIL', # Utilities: Electric and Gas Utilities, Jan 1939 to Sep 2022
         'IPG311A2S', # Manufacturing: Non-Durable Goods: Food, Beverage, and Tobacco, Jan 1972 to Sep 2022 
         'WPU058103', # Non-Durable Goods: Petroleum and Coal Products, Jul 1985 to Sep 2022
    
         'WPS061', # PPI: Industrial Chemicals and Allied Products, Jan 1967 to Sep 2022
         'PCE', # Personal Consumption Expenditure, Jan 1959 to Aug 2022
         'PCU311311', # PPI: Food manufacturing, Dec 1984 to Sep 2022
        'WPU083', # PPI: Lumber and Wood Products: Plywood, Jan 1947 to Sep 2022 
        'WPSFD4111', # Final demand: finished consumer goods, Apr 1947 to Sep 2022
        'USAGDPDEFQISMEI', # GDP Deflator, Q1 1960 to Q2 2022
        'AHETPI', # Average Hourly Earnings of Production and Nonsupervisory Employees, Jan 1964 to Sep 2022
        'CPIAUCSL', # CPI: All items in US city, Jan 1947 to Sep 2022
        'A829RD3Q086SBEA', # Government consumption expenditures and gross investment: State and local 
                           # (implicit price deflator), Q1 1947 to Q2 2022
        'CSUSHPISA', # S&P/Case-Shiller U.S. National Home Price Index,  Jan 1987 to Jul 2022
        'PDI', # Personal Income Receipts on Assets: Personal Dividend Income, Jan 1959 to Aug 2022
        'NASDAQCOM', # NASDAQ Composite Index, 1971-02-05 to 2022-10-17
    
    # spot exchange rate: x currency to USD at the spot market.
    
        'DEXUSUK', # U.K. Pound Sterling Spot Exchange Rate, 1971-01-04 to 2022-10-14
        'DEXJPUS', # Japanese yen, 1971-01-04 to 2022-10-14
        'DEXCAUS', # Canadian dollar, 1971-01-04 to 2022-09-16
        'RNUSBIS', # Real Narrow Effective Exchange Rate, Jan 1964 to Aug 2022
        'DEXCHUS', # Chinese renmibi yuan, 1981-01-02 to 2022-10-14
        'CCUSMA02AUM618N', # Australian dollar, Jan 1960 to Aug 2022
    
    # Treasury yields of various maturities
    
       
        'DGS3MO', # 3 months, 
        'DGS1',  # 1 year
        'DGS10', # 10 years
    
    
    
        'UNRATE', # Unemployment rate, Jan 1948 to Sep 2022 
        'EMRATIO', # Employment-Population Ratio, Jan 1948 to Sep 2022
    
     #   Equity Market Volatility
    
        'EMVOVERALLEMV', # Overall, Jan 1985 to Aug 2022
        'EMVCOMMMKT', # Commodity markets, Jan 1985 to Aug 2022
    
        'TLAACBW027SBOG', # Total Assets, All Commercial Banks, 1973-01-03 to 2022-10-05
        'DFF'] # federal funds rate, 1954-07-01 to 2022-10-13
 

# as some variables have daily data, such as ffr, and 10 year Treasury yield, 
# data2 will have daily observations for these two, with NaN for the monthly variables
data4 = DataReader(macro4,"fred" ,start=datetime(1987, 1, 1), end=datetime(2022, 9, 1))
         
data4.columns = ['govt_exp', 'houst_st', 'oil_brent', 'oil_wti', 'ui', 'rgdp', 'manuf', 'transport', 'ipi',
                '30ymortgage', 'employ_goods', 'electric_gas','food_bev', 'petro_coal', 'indust_chemical', 
                 'pce', 'food_manuf', 'lumber_wood', 'finished_goods', 'gdp_deflator', 'hour_earn', 'cpi', 
                'govt_invest', 'hpi', 'dividend', 'nasdaq', 'pound_usd', 'yen_usd', 'can_usd', 'rneer','yuan_usd' ,
                 'aus_usd','tres3mo',  'tres1y','tres10y','unrate','epop' ,'overall_vol', 'commod_vol',
                 'assets', 'ffr'] 

data4.head()

,govt_exp,houst_st,oil_brent,oil_wti,ui,rgdp,manuf,transport,ipi,30ymortgage,...,aus_usd,tres3mo,tres1y,tres10y,unrate,epop,overall_vol,commod_vol,assets,ffr
DATE,,,,,,,,,,,,,,,,,,,,,
1987-01-01,1045.149,1774.0,NaN,18.656,NaN,8375.274,17465.0,3098.3,56.1259,NaN,...,1.51629,NaN,NaN,NaN,6.6,61.0,15.68098,8.85025,NaN,14.35
1987-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.37,...,NaN,5.70,5.86,7.18,NaN,NaN,NaN,NaN,NaN,6.46
1987-01-03,NaN,NaN,NaN,NaN,2590000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.46
1987-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.46
1987-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.71,5.85,7.08,NaN,NaN,NaN,NaN,NaN,6.86


In [6]:
# gather the data for the first day of each month to store the monthly data from daily data
# groupby the month and then get the zeroth (nth) row of each group.
data4.sort_index(inplace=True)
data_grouped4 = data4.groupby(pd.Grouper(freq ='M')).nth(0)

# spline interpolation method to fill the NaN
inter_data4 = data_grouped4.interpolate(method='spline', order =3).dropna()
inter_data4.head()

,govt_exp,houst_st,oil_brent,oil_wti,ui,rgdp,manuf,transport,ipi,30ymortgage,...,aus_usd,tres3mo,tres1y,tres10y,unrate,epop,overall_vol,commod_vol,assets,ffr
DATE,,,,,,,,,,,,,,,,,,,,,
1987-08-31,1071.806147,1605.0,19.230085,20.274,2.232000e+06,8590.005245,17630.0,3172.5,58.8171,10.233336,...,1.417190,6.481263,7.467720,8.785167,6.0,61.8,12.10832,6.73264,2694.870900,6.75
1987-09-30,1081.600983,1695.0,18.430000,19.526,2.176455e+06,8642.989184,17691.0,3192.4,59.0391,10.212912,...,1.376957,6.320000,7.300000,9.050000,5.9,61.6,11.07046,5.29031,2710.552476,6.84
1987-10-31,1089.898000,1515.0,18.500000,19.848,2.128274e+06,8685.694000,17729.0,3202.9,59.8340,10.191791,...,1.404909,6.860000,7.940000,9.660000,6.0,61.8,55.27221,27.11467,2728.768548,7.71
1987-11-30,1098.374613,1656.0,17.651648,18.920,2.089959e+06,8705.004928,17775.0,3210.8,60.1367,10.171334,...,1.459810,6.895969,7.708123,8.716398,5.8,61.9,54.93812,26.11812,2747.687426,6.62
1987-12-31,1103.107959,1400.0,17.650000,17.241,2.058318e+06,8714.332664,17809.0,3213.6,60.4739,10.150178,...,1.408448,5.720000,7.050000,9.010000,5.7,62.0,37.79671,18.05297,2767.884575,7.04


In [55]:
inter_data4.to_excel("data_favar.xlsx",index= True, header= True)

In [7]:
def adfuller_test(series, signif = 0.05, name = '', verbose = False):
    """Perform ADFuller to test for stationarity of a given series and print report"""
    
    r = adfuller(series, autolag = 'AIC')
    output = {'test_statistic': round(r[0], 4), 'pvalue': round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue']
    def adjust(val, length = 6): return str(val).ljust(length)
    
    # print sumamry
    print(f'     Augmented Dickey-Fuller Test on "{name}"', "\n    ", '-'*47)
    print(f'  Null Hypothesis: Data has unit too. Non-Stationary.')
    print(f'  Significance Level    = {signif}')
    print(f'  Test Statistic        = {output["test_statistic"]}')
    print(f'  No. of Lags Chosen    = {output["n_lags"]}')
          
    for key, val in r[4].items():
          print(f'  Critical Value {adjust(key)} = {round(val, 3)}')
          
    if p_value <= signif:
          print(f" => P-Value = {p_value}. Rejecting null hypothesis.")
          print(f" => Series is stationary.")
    else:
          print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
          print(f" => Series is non-stationary.")
          
     # ADF Test on each column
for name, column in inter_data4.iteritems():
          adfuller_test(column, name = column.name)
          print('\n')

     Augmented Dickey-Fuller Test on "govt_exp" 
     -----------------------------------------------
  Null Hypothesis: Data has unit too. Non-Stationary.
  Significance Level    = 0.05
  Test Statistic        = 0.3991
  No. of Lags Chosen    = 18
  Critical Value 1%     = -3.447
  Critical Value 5%     = -2.869
  Critical Value 10%    = -2.571
 => P-Value = 0.9815. Weak evidence to reject the Null Hypothesis.
 => Series is non-stationary.


     Augmented Dickey-Fuller Test on "houst_st" 
     -----------------------------------------------
  Null Hypothesis: Data has unit too. Non-Stationary.
  Significance Level    = 0.05
  Test Statistic        = -2.4344
  No. of Lags Chosen    = 17
  Critical Value 1%     = -3.447
  Critical Value 5%     = -2.869
  Critical Value 10%    = -2.571
 => P-Value = 0.1323. Weak evidence to reject the Null Hypothesis.
 => Series is non-stationary.


     Augmented Dickey-Fuller Test on "oil_brent" 
     -----------------------------------------------
  

     Augmented Dickey-Fuller Test on "pound_usd" 
     -----------------------------------------------
  Null Hypothesis: Data has unit too. Non-Stationary.
  Significance Level    = 0.05
  Test Statistic        = -1.2187
  No. of Lags Chosen    = 7
  Critical Value 1%     = -3.446
  Critical Value 5%     = -2.869
  Critical Value 10%    = -2.571
 => P-Value = 0.6655. Weak evidence to reject the Null Hypothesis.
 => Series is non-stationary.


     Augmented Dickey-Fuller Test on "yen_usd" 
     -----------------------------------------------
  Null Hypothesis: Data has unit too. Non-Stationary.
  Significance Level    = 0.05
  Test Statistic        = -2.7193
  No. of Lags Chosen    = 11
  Critical Value 1%     = -3.446
  Critical Value 5%     = -2.869
  Critical Value 10%    = -2.571
 => P-Value = 0.0708. Weak evidence to reject the Null Hypothesis.
 => Series is non-stationary.


     Augmented Dickey-Fuller Test on "can_usd" 
     -----------------------------------------------
  Nu

The stationary variables are 30ymortgage, yuan_usd, unrate, commod_vol, and ffr